In [66]:
import pandas as pd

# 1. Đường dẫn đến "ngôi nhà" của dữ liệu
# Dấu r ở phía trước giúp máy tính không bị nhầm lẫn với các ký tự đặc biệt
duong_dan = r'../Data_goc/CA_Weather_Fire_Dataset_1984-2025.csv'

# 2. Nhờ Pandas đọc file này
try:
    df = pd.read_csv(duong_dan)

    # 3. Đếm xem có bao nhiêu "ô trống" (NaN) trong toàn bộ bảng
    tong_o_trong = df.isnull().sum().sum()
    print(f"tổng cộng có {tong_o_trong} ô trống nhé!")

    # 4. Tìm vị trí các ô trống và in ra bảng
    # Chúng mình sẽ tạo một danh sách các vị trí (Dòng nào, Cột nào)
    vi_tri_nan = df.isnull().stack()
    vi_tri_nan = vi_tri_nan[vi_tri_nan].index.tolist()

    print("\nĐây là bảng vị trí các ô trống (Dòng, Cột):")
    for vt in vi_tri_nan:
        print(f"- Dòng {vt[0]} ở cột '{vt[1]}'")

except FileNotFoundError:
    print("không tìm thấy file ở đường dẫn đó rồi kiểm tra lại nhé.")

tổng cộng có 28 ô trống nhé!

Đây là bảng vị trí các ô trống (Dòng, Cột):
- Dòng 4808 ở cột 'AVG_WIND_SPEED'
- Dòng 4808 ở cột 'WIND_TEMP_RATIO'
- Dòng 4809 ở cột 'AVG_WIND_SPEED'
- Dòng 4809 ở cột 'WIND_TEMP_RATIO'
- Dòng 4810 ở cột 'AVG_WIND_SPEED'
- Dòng 4810 ở cột 'WIND_TEMP_RATIO'
- Dòng 14879 ở cột 'AVG_WIND_SPEED'
- Dòng 14879 ở cột 'WIND_TEMP_RATIO'
- Dòng 14971 ở cột 'AVG_WIND_SPEED'
- Dòng 14971 ở cột 'WIND_TEMP_RATIO'
- Dòng 14972 ở cột 'AVG_WIND_SPEED'
- Dòng 14972 ở cột 'WIND_TEMP_RATIO'
- Dòng 14973 ở cột 'AVG_WIND_SPEED'
- Dòng 14973 ở cột 'WIND_TEMP_RATIO'
- Dòng 14974 ở cột 'AVG_WIND_SPEED'
- Dòng 14974 ở cột 'WIND_TEMP_RATIO'
- Dòng 14975 ở cột 'PRECIPITATION'
- Dòng 14975 ở cột 'MAX_TEMP'
- Dòng 14975 ở cột 'MIN_TEMP'
- Dòng 14975 ở cột 'AVG_WIND_SPEED'
- Dòng 14975 ở cột 'TEMP_RANGE'
- Dòng 14975 ở cột 'WIND_TEMP_RATIO'
- Dòng 14976 ở cột 'AVG_WIND_SPEED'
- Dòng 14976 ở cột 'WIND_TEMP_RATIO'
- Dòng 14986 ở cột 'AVG_WIND_SPEED'
- Dòng 14986 ở cột 'WIND_TEMP_RATIO'
- 

như đã thấy thì ta có các trường thiếu dữ liệu đó là PRECIPITATION, MAX_TEMP, MIN_TEMP, AVG_WIND_SPEED, TEMP_RANGE, WIND_TEMP_RATIO	

# Phân loại phương pháp

PRECIPITATION và AVG_WIND_SPEED : dùng pp nội suy + chặn dưới 0
Vì mưa là hiện tượng thời tiết có tính xu hướng nên dùng thuật toán nội suy đơn thuần chỉ là toán học + mưa và tốc độ gió không thể âm
ví dụ
Ngày 1: 0mm (Có dữ liệu)
Ngày 2: (Thiếu)
Ngày 3: (Thiếu)
Ngày 4: (Thiếu)
Ngày 5: (Thiếu)
Ngày 6: (Thiếu)
Ngày 7: 10mm (Có dữ liệu)
có 6 bước nhảy 10/6 = 1.67
nên mỗi ngày sẽ tăng lên 1.67

TEMP_RANGE và WIND_TEMP_RATIO là biến dẫn xuất KHÔNG PHẢI dữ liệu đo đạc trực tiếp nên nếu dùng nội suy dữ liệu sẽ bị sai lệch toán học<br>
thay vào đó ta sẽ tính lại thông qua các trường khác<br>
TEMP_RANGE = MAX_TEMP - MIN_TEMP <br>
WIND_TEMP_RATIO = AVG_WIND_SPEED/MAX_TEMP (ko loại trừ trường hợp nhiệt độ MAX_TEMP có thể về 0 thay vào đó ta dùng 0.0001 để không gây ra lỗi)<br>

MAX_TEMP, MIN_TEMP  : dùng pp Nội suy tuyến tính (Linear Interpolation) giống PRECIPITATION và AVG_WIND_SPEED chỉ khác là nhiệt độ có thể âm

# BẮT BUỘC

THEO thứ tự tính toán<br>
1 tính MAX_TEMP, MIN_TEMP dùng pp nội suy tuyến tính<br>
2 tính PRECIPITATION và AVG_WIND_SPEED dùng pp nội suy tuyến tính + chặn dưới 0 trước<br>
3 tính TEMP_RANGE và WIND_TEMP_RATIO dùng pp tính toán lại<br>

# LÍ DO

Lí do là tạo ra thứ tự tính toán để phòng trường hợp 1 hàng có nhiều giá trị nan như dòng 14975 thiếu 6 giá trị PRECIPITATION,MAX_TEMP,MIN_TEMP,AVG_WIND_SPEED ,TEMP_RANGE và WIND_TEMP_RATIO(trường hợp xấu nhất có thể xảy ra)

các dữ liệu như TEMP_RANGE , WIND_TEMP_RATIO tính được "THÔNG QUA" các dữ liệu khác như MAX_TEMP - MIN_TEMP và AVG_WIND_SPEED/MAX_TEMP nên phải tính các giá trị này trước

# CODE

### Bước 1 đọc file CA_Weather_Fire_Dataset_1984-2025.csv tìm NaN của MAX_TEMP, MIN_TEMP in ra vị trí mà 2 trường này đang thiếu(NaN)

In [67]:
import pandas as pd

# 1. Đọc file dữ liệu
df = pd.read_csv("../Data_goc/CA_Weather_Fire_Dataset_1984-2025.csv")

# (Tùy chọn) Chuyển đổi cột DATE để nhìn cho dễ, chưa can thiệp dữ liệu
df['DATE'] = pd.to_datetime(df['DATE'])

# 2. Tìm các dòng bị thiếu (NaN)
nan_max = df[df['MAX_TEMP'].isnull()]
nan_min = df[df['MIN_TEMP'].isnull()]

# 3. In ra kết quả kiểm tra
print(f"--- BÁO CÁO KIỂM TRA DỮ LIỆU THIẾU (BƯỚC 1) ---")

# Kiểm tra MAX_TEMP
print(f"\n1. Cột MAX_TEMP: Có {len(nan_max)} dòng bị thiếu.")
if len(nan_max) > 0:
    print(">> Chi tiết vị trí thiếu MAX_TEMP:")
    # In ra index, ngày và giá trị để đối chiếu
    display(nan_max[['DATE', 'MAX_TEMP', 'MIN_TEMP']])

print("-" * 50)

# Kiểm tra MIN_TEMP
print(f"\n2. Cột MIN_TEMP: Có {len(nan_min)} dòng bị thiếu.")
if len(nan_min) > 0:
    print(">> Chi tiết vị trí thiếu MIN_TEMP:")
    display(nan_min[['DATE', 'MAX_TEMP', 'MIN_TEMP']])

--- BÁO CÁO KIỂM TRA DỮ LIỆU THIẾU (BƯỚC 1) ---

1. Cột MAX_TEMP: Có 1 dòng bị thiếu.
>> Chi tiết vị trí thiếu MAX_TEMP:


,DATE,MAX_TEMP,MIN_TEMP
14975,2024-12-31,NaN,NaN


--------------------------------------------------

2. Cột MIN_TEMP: Có 1 dòng bị thiếu.
>> Chi tiết vị trí thiếu MIN_TEMP:


,DATE,MAX_TEMP,MIN_TEMP
14975,2024-12-31,NaN,NaN


### Bước 2 dùng phương pháp nội suy tuyến tính tính ra và thay thế vào vị trí NaN đó rồi lưu vào file  mới tên CA_Weather_Fire_Dataset_1984-2025_clean.csv 

In [68]:
# --- BƯỚC 2: NỘI SUY DỮ LIỆU & LƯU FILE MỚI ---

print(">> Đang thực hiện nội suy tuyến tính cho MAX_TEMP và MIN_TEMP...")

# 1. Áp dụng phương pháp Nội suy tuyến tính (Linear Interpolation)
# method='linear': Nối đường thẳng giữa 2 điểm có dữ liệu để tìm điểm thiếu
df['MAX_TEMP'] = df['MAX_TEMP'].interpolate(method='linear')
df['MIN_TEMP'] = df['MIN_TEMP'].interpolate(method='linear')

# 2. Kiểm tra lại xem còn NaN không (để chắc chắn đã điền xong)
remaining_nan = df[['MAX_TEMP', 'MIN_TEMP']].isnull().sum().sum()
if remaining_nan == 0:
    print("✅ Đã lấp đầy toàn bộ dữ liệu thiếu cho MAX_TEMP và MIN_TEMP.")
else:
    print(f"⚠️ Vẫn còn {remaining_nan} dòng chưa thể nội suy (có thể do nằm ở đầu/cuối file).")

# 3. Lưu vào file mới
output_filename = "CA_Weather_Fire_Dataset_1984-2025_clean.csv"
df.to_csv(output_filename, index=False)

print(f"\n💾 Đã lưu file kết quả: {output_filename}")

# (Tùy chọn) In lại đúng các dòng lúc nãy bị thiếu để xem kết quả máy tính điền
# Sử dụng lại biến nan_max, nan_min từ Bước 1 nếu bạn chạy cùng phiên làm việc
try:
    if len(nan_max) > 0:
        print("\n>> Kiểm tra lại các dòng MAX_TEMP vừa được điền:")
        # Lấy lại các index cũ để xem dữ liệu mới
        filled_indices = nan_max.index
        display(df.loc[filled_indices, ['DATE', 'MAX_TEMP', 'MIN_TEMP']].head())
except NameError:
    pass # Bỏ qua nếu bạn không chạy Bước 1 ngay trước đó

>> Đang thực hiện nội suy tuyến tính cho MAX_TEMP và MIN_TEMP...
✅ Đã lấp đầy toàn bộ dữ liệu thiếu cho MAX_TEMP và MIN_TEMP.

💾 Đã lưu file kết quả: CA_Weather_Fire_Dataset_1984-2025_clean.csv

>> Kiểm tra lại các dòng MAX_TEMP vừa được điền:


,DATE,MAX_TEMP,MIN_TEMP
14975,2024-12-31,60.5,49.5


### Bước 3 đọc file CA_Weather_Fire_Dataset_1984-2025_clean.csv và tìm NaN của PRECIPITATION và AVG_WIND_SPEED in ra vị trí mà 2 trường này đang thiếu(NaN)

In [69]:
import pandas as pd

# 1. Đọc file "Clean" đã lưu ở Bước 2
df = pd.read_csv("CA_Weather_Fire_Dataset_1984-2025_clean.csv")

# Chuyển định dạng ngày để hiển thị cho dễ nhìn
df['DATE'] = pd.to_datetime(df['DATE'])

# 2. Tìm các dòng bị thiếu (NaN) cho Mưa và Gió
nan_precip = df[df['PRECIPITATION'].isnull()]
nan_wind = df[df['AVG_WIND_SPEED'].isnull()]

# 3. In kết quả kiểm tra
print(f"--- BÁO CÁO KIỂM TRA DỮ LIỆU THIẾU (BƯỚC 3) ---")

# Kiểm tra PRECIPITATION
print(f"\n1. Cột PRECIPITATION (Lượng mưa): Có {len(nan_precip)} dòng bị thiếu.")
if len(nan_precip) > 0:
    print(">> Chi tiết vị trí thiếu Mưa:")
    # Hiển thị các cột liên quan để dễ đối chiếu
    display(nan_precip[['DATE', 'PRECIPITATION', 'AVG_WIND_SPEED']])

print("-" * 50)

# Kiểm tra AVG_WIND_SPEED
print(f"\n2. Cột AVG_WIND_SPEED (Tốc độ gió): Có {len(nan_wind)} dòng bị thiếu.")
if len(nan_wind) > 0:
    print(">> Chi tiết vị trí thiếu Gió:")
    display(nan_wind[['DATE', 'PRECIPITATION', 'AVG_WIND_SPEED']])

--- BÁO CÁO KIỂM TRA DỮ LIỆU THIẾU (BƯỚC 3) ---

1. Cột PRECIPITATION (Lượng mưa): Có 1 dòng bị thiếu.
>> Chi tiết vị trí thiếu Mưa:


,DATE,PRECIPITATION,AVG_WIND_SPEED
14975,2024-12-31,NaN,NaN


--------------------------------------------------

2. Cột AVG_WIND_SPEED (Tốc độ gió): Có 12 dòng bị thiếu.
>> Chi tiết vị trí thiếu Gió:


,DATE,PRECIPITATION,AVG_WIND_SPEED
4808,1997-03-01,0.0,NaN
4809,1997-03-02,0.0,NaN
4810,1997-03-03,0.0,NaN
14879,2024-09-26,0.0,NaN
14971,2024-12-27,0.0,NaN
14972,2024-12-28,0.0,NaN
14973,2024-12-29,0.0,NaN
14974,2024-12-30,0.0,NaN
14975,2024-12-31,NaN,NaN
14976,2025-01-01,0.0,NaN


### Bước 4 dùng phương pháp nội suy tuyến tính + chặn dưới 0  tính ra và thay thế vào vị trí NaN đó rồi lưu lại

In [70]:
# --- BƯỚC 4: NỘI SUY + CHẶN DƯỚI 0 & CẬP NHẬT FILE ---

print(">> Đang xử lý dữ liệu Mưa (PRECIPITATION) và Gió (AVG_WIND_SPEED)...")

# 1. Nội suy tuyến tính (Linear Interpolation)
# Điền dữ liệu vào chỗ trống dựa trên xu hướng
df['PRECIPITATION'] = df['PRECIPITATION'].interpolate(method='linear')
df['AVG_WIND_SPEED'] = df['AVG_WIND_SPEED'].interpolate(method='linear')

# 2. Chặn dưới 0 (Clipping)
# Quy tắc vật lý: Mưa và Gió không thể là số âm.
# Lệnh này sẽ biến mọi số âm (nếu có do nội suy) thành số 0.
df['PRECIPITATION'] = df['PRECIPITATION'].clip(lower=0)
df['AVG_WIND_SPEED'] = df['AVG_WIND_SPEED'].clip(lower=0)

# 3. Kiểm tra lại kết quả
remaining_nan = df[['PRECIPITATION', 'AVG_WIND_SPEED']].isnull().sum().sum()
if remaining_nan == 0:
    print("✅ Đã xử lý sạch sẽ Mưa và Gió (Không còn NaN, Không có số âm).")
else:
    print(f"⚠️ Vẫn còn {remaining_nan} dòng chưa thể điền (thường là các dòng đầu tiên của file).")

# 4. Lưu đè kết quả vào file clean
output_filename = "CA_Weather_Fire_Dataset_1984-2025_clean.csv"
df.to_csv(output_filename, index=False)

print(f"\n💾 Đã cập nhật và lưu file: {output_filename}")

# (Tùy chọn) Kiểm tra thử một vài dòng xem có số âm không
print(">> Kiểm tra nhanh giá trị nhỏ nhất (Min) để đảm bảo không âm:")
print(f"- Min Precipitation: {df['PRECIPITATION'].min()}")
print(f"- Min Wind Speed: {df['AVG_WIND_SPEED'].min()}")

>> Đang xử lý dữ liệu Mưa (PRECIPITATION) và Gió (AVG_WIND_SPEED)...
✅ Đã xử lý sạch sẽ Mưa và Gió (Không còn NaN, Không có số âm).

💾 Đã cập nhật và lưu file: CA_Weather_Fire_Dataset_1984-2025_clean.csv
>> Kiểm tra nhanh giá trị nhỏ nhất (Min) để đảm bảo không âm:
- Min Precipitation: 0.0
- Min Wind Speed: 1.79


### Bước 5 đọc file CA_Weather_Fire_Dataset_1984-2025_clean.csv và tìm NaN của TEMP_RANGE và WIND_TEMP_RATIO in ra vị trí mà 2 trường này đang thiếu(NaN)

In [71]:
import pandas as pd

# 1. Đọc file "Clean" (đã xử lý Mưa/Gió/Nhiệt ở Bước 4)
df = pd.read_csv("CA_Weather_Fire_Dataset_1984-2025_clean.csv")

# Chuyển đổi định dạng ngày
df['DATE'] = pd.to_datetime(df['DATE'])

# 2. Tìm các dòng bị thiếu (NaN) cho Range và Ratio
nan_range = df[df['TEMP_RANGE'].isnull()]
nan_ratio = df[df['WIND_TEMP_RATIO'].isnull()]

# 3. In kết quả kiểm tra
print(f"--- BÁO CÁO KIỂM TRA DỮ LIỆU THIẾU (BƯỚC 5) ---")

# Kiểm tra TEMP_RANGE
print(f"\n1. Cột TEMP_RANGE (Biên độ nhiệt): Có {len(nan_range)} dòng bị thiếu.")
if len(nan_range) > 0:
    print(">> Chi tiết vị trí thiếu TEMP_RANGE:")
    # Hiển thị cả MAX và MIN để thấy rằng: Dù MAX/MIN đã có số (nhờ Bước 2), nhưng RANGE vẫn đang NaN
    display(nan_range[['DATE', 'MAX_TEMP', 'MIN_TEMP', 'TEMP_RANGE']].head(10))

print("-" * 50)

# Kiểm tra WIND_TEMP_RATIO
print(f"\n2. Cột WIND_TEMP_RATIO (Tỷ lệ Gió/Nhiệt): Có {len(nan_ratio)} dòng bị thiếu.")
if len(nan_ratio) > 0:
    print(">> Chi tiết vị trí thiếu WIND_TEMP_RATIO:")
    # Hiển thị Gió và Nhiệt để đối chiếu
    display(nan_ratio[['DATE', 'AVG_WIND_SPEED', 'MAX_TEMP', 'WIND_TEMP_RATIO']].head(10))

--- BÁO CÁO KIỂM TRA DỮ LIỆU THIẾU (BƯỚC 5) ---

1. Cột TEMP_RANGE (Biên độ nhiệt): Có 1 dòng bị thiếu.
>> Chi tiết vị trí thiếu TEMP_RANGE:


,DATE,MAX_TEMP,MIN_TEMP,TEMP_RANGE
14975,2024-12-31,60.5,49.5,NaN


--------------------------------------------------

2. Cột WIND_TEMP_RATIO (Tỷ lệ Gió/Nhiệt): Có 12 dòng bị thiếu.
>> Chi tiết vị trí thiếu WIND_TEMP_RATIO:


,DATE,AVG_WIND_SPEED,MAX_TEMP,WIND_TEMP_RATIO
4808,1997-03-01,12.417500,64.0,NaN
4809,1997-03-02,10.515000,60.0,NaN
4810,1997-03-03,8.612500,59.0,NaN
14879,2024-09-26,6.930000,70.0,NaN
14971,2024-12-27,4.667143,61.0,NaN
14972,2024-12-28,4.634286,59.0,NaN
14973,2024-12-29,4.601429,58.0,NaN
14974,2024-12-30,4.568571,62.0,NaN
14975,2024-12-31,4.535714,60.5,NaN
14976,2025-01-01,4.502857,59.0,NaN


### Bước 6 dùng phương pháp tính toán lại tính ra và thay thế vào vị trí NaN đó rồi lưu lại

In [72]:
# --- BƯỚC 6: TÍNH TOÁN LẠI BIẾN DẪN XUẤT & CẬP NHẬT FILE ---

print(">> Đang tính toán lại TEMP_RANGE và WIND_TEMP_RATIO...")

# 1. Logic An toàn (Safety Logic): Đảm bảo MAX >= MIN trước khi trừ
# Nếu dòng nào Max < Min (do lỗi nhập liệu), ta đảo ngược giá trị lại
swap_mask = df['MAX_TEMP'] < df['MIN_TEMP']
if swap_mask.sum() > 0:
    print(f"⚠️ Phát hiện {swap_mask.sum()} dòng vô lý (Max < Min). Đang hoán đổi lại cho đúng logic...")
    # Hoán đổi
    temp_vals = df.loc[swap_mask, 'MAX_TEMP']
    df.loc[swap_mask, 'MAX_TEMP'] = df.loc[swap_mask, 'MIN_TEMP']
    df.loc[swap_mask, 'MIN_TEMP'] = temp_vals

# 2. TÍNH TOÁN LẠI (RE-CALCULATE)
# Thay vì điền vào chỗ trống, ta tính lại toàn bộ cột để đảm bảo đúng 100% với dữ liệu gốc

# 2.1. TEMP_RANGE = Nhiệt độ cao nhất - Nhiệt độ thấp nhất
df['TEMP_RANGE'] = df['MAX_TEMP'] - df['MIN_TEMP']

# 2.2. WIND_TEMP_RATIO = Tốc độ gió / Nhiệt độ cao nhất
# Cộng thêm 0.1 vào mẫu số để tránh lỗi chia cho 0 (nếu Max Temp = 0)
df['WIND_TEMP_RATIO'] = df['AVG_WIND_SPEED'] / (df['MAX_TEMP'] + 0.1)

# 3. Kiểm tra kết quả
# Nếu logic đúng, sẽ không còn NaN nào cả
nan_check = df[['TEMP_RANGE', 'WIND_TEMP_RATIO']].isnull().sum().sum()

if nan_check == 0:
    print("✅ Đã tính toán xong. TEMP_RANGE và WIND_TEMP_RATIO đã đầy đủ 100%.")
else:
    print(f"⚠️ Vẫn còn {nan_check} dòng bị lỗi (cần kiểm tra lại đầu vào).")

# 4. Lưu đè kết quả vào file clean
output_filename = "../Data_clean/CA_Weather_Fire_Dataset_1984-2025_clean.csv"
df.to_csv(output_filename, index=False)

print(f"\n💾 Đã cập nhật và lưu file: {output_filename}")

# (Tùy chọn) Kiểm tra thử một dòng để chứng minh phép tính đúng
# Lấy dòng đầu tiên để test
sample = df.iloc[0]
print("\n>> Kiểm tra logic phép tính tại dòng đầu tiên:")
print(f"- Max: {sample['MAX_TEMP']:.2f}, Min: {sample['MIN_TEMP']:.2f}, Gió: {sample['AVG_WIND_SPEED']:.2f}")
print(f"- Range (Máy tính): {sample['TEMP_RANGE']:.2f} | Range (Thực tế Max-Min): {sample['MAX_TEMP'] - sample['MIN_TEMP']:.2f}")
print(f"- Ratio (Máy tính): {sample['WIND_TEMP_RATIO']:.4f}")

>> Đang tính toán lại TEMP_RANGE và WIND_TEMP_RATIO...
✅ Đã tính toán xong. TEMP_RANGE và WIND_TEMP_RATIO đã đầy đủ 100%.

💾 Đã cập nhật và lưu file: ../Data_clean/CA_Weather_Fire_Dataset_1984-2025_clean.csv

>> Kiểm tra logic phép tính tại dòng đầu tiên:
- Max: 79.00, Min: 51.00, Gió: 4.70
- Range (Máy tính): 28.00 | Range (Thực tế Max-Min): 28.00
- Ratio (Máy tính): 0.0594


### ktra lại file mới đã clean chưa

In [73]:
# --- BƯỚC 7: KIỂM TRA TOÀN DIỆN FINAL (FINAL CHECK) ---
import pandas as pd

# 1. Đọc file kết quả từ Bước 6
df = pd.read_csv("../Data_clean/CA_Weather_Fire_Dataset_1984-2025_clean.csv")

# 2. Đếm số lượng NaN ở TẤT CẢ các cột
null_counts = df.isnull().sum()

# Lọc ra những cột nào còn chứa NaN (lớn hơn 0)
cols_with_nan = null_counts[null_counts > 0]

print(f"--- BÁO CÁO TỔNG KẾT DỮ LIỆU SẠCH ---")
print(f"Tổng số dòng dữ liệu: {len(df)}")

if len(cols_with_nan) == 0:
    print("\n✅ CHÚC MỪNG! File dữ liệu đã sạch bong (Không còn bất kỳ ô trống nào).")
else:
    print(f"\n⚠️ Vẫn còn {len(cols_with_nan)} cột chứa dữ liệu thiếu:")
    print("-" * 40)
    print(f"{'Tên Cột':<25} | {'Số lượng thiếu':<15} | {'Tỷ lệ %'}")
    print("-" * 40)
    
    for col_name, count in cols_with_nan.items():
        percent = (count / len(df)) * 100
        print(f"{col_name:<25} | {count:<15} | {percent:.2f}%")
    
    print("-" * 40)
    
    # Gợi ý xử lý dựa trên tên cột còn thiếu
    print("\n>> ĐỀ XUẤT XỬ LÝ CHO CÁC CỘT CÒN LẠI:")
    if 'FIRE_START_DAY' in cols_with_nan:
        print("- Với 'FIRE_START_DAY' (Target): Bắt buộc phải XÓA các dòng này (vì không có đáp án để học).")
    
    lag_cols = [c for c in cols_with_nan.index if 'LAGGED' in c]
    if len(lag_cols) > 0:
        print(f"- Với các cột 'LAGGED...' ({len(lag_cols)} cột): Nên XÓA BỎ các cột này (Drop columns).")
        print("  (Lý do: Dữ liệu Lag cũ chứa sai số, ta sẽ tạo lại Lag mới từ dữ liệu sạch trong bước Feature Engineering).")

--- BÁO CÁO TỔNG KẾT DỮ LIỆU SẠCH ---
Tổng số dòng dữ liệu: 14988

✅ CHÚC MỪNG! File dữ liệu đã sạch bong (Không còn bất kỳ ô trống nào).
